# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# Dataset

In [ ]:
data = {
    'Country': ['FRA', 'GER', 'GBR', 'ITA', 'SPA', 'USA', 'JAP', 'CAN'],
    'GDP': [133.40, 138.80, 125.10, 120.20, 92.50, 176.20, 142.00, 166.70],
    'INF': [3.00, 3.40, 6.30, 7.60, 7.30, 4.30, 2.20, 3.30],
    'DEF': [-1.50, -1.90, -1.30, -11.50, -3.60, -2.50, 2.90, -4.10],
    'DEB': [46.60, 43.60, 34.70, 100.50, 46.80, 56.20, 69.80, 71.90],
    'INT': [10.40, 6.00, 11.10, 11.90, 14.70, 8.70, 7.40, 10.80],
    'TRB': [-2.10, 5.90, -4.00, -0.70, -6.50, -2.70, 1.90, 1.60],
    'UNE': [8.90, 6.20, 6.80, 11.20, 15.90, 5.50, 2.10, 8.10]
}

df = pd.DataFrame(data)
df.set_index('Country', inplace=True)

df

# Algorithm

In [ ]:
class TrimmedBlockCEM:
    def __init__(self):
        pass

    def auto_apply(self, df, cluster, alpha, epochs=10 , max_iter=100, tol=1, initialization_theta='uniform', initialization_cluster='random'):
        best_try = {'log_likelihoods' : [-np.inf]}
        n, p = df.shape
        g, m = cluster
        for epoch in range(epochs):
            df_shuffled = df.sample(frac=1)
            df_shuffled = df_shuffled.sample(frac=1, axis=1)
            X = df_shuffled.values
            X = (X - X.mean(axis=0)) / X.std(axis=0)

            mus = np.zeros((g, m))
            sigmas = np.zeros((g, m))
            block_lignes = n//g
            block_colonnes = p//m
            for k in range(g):
                for l in range(m):
                    mus[k,l] = np.mean(X[k*block_lignes:(k+1)*block_lignes, l*block_colonnes:(l+1)*block_colonnes])
                    sigmas[k,l] = np.var(X[k*block_lignes:(k+1)*block_lignes, l*block_colonnes:(l+1)*block_colonnes])

            try:
                res = self.apply(X, mus, sigmas, cluster, alpha,  max_iter, tol, initialization_theta, initialization_cluster)
                print('Epoch', epoch, 'log_likelihood', res['log_likelihoods'][-1])
                if res['log_likelihoods'][-1] > best_try['log_likelihoods'][-1]:
                    best_try = res
                    best_try['lignes'] = df_shuffled.index
                    best_try['colonnes'] = df_shuffled.columns
            except ValueError as e:
                #print(e)
                pass

        
        return best_try


    def apply(self, X, mus, sigmas, cluster, alpha,  max_iter=100, tol=1, initialization_theta='uniform', initialization_cluster='random'):
        if mus.shape[0]==sigmas.shape[0]==cluster[0] and mus.shape[1]==sigmas.shape[1]==X.shape[1]:
            raise ValueError('The number of cluster and the number of rows of mus and sigmas must be the same')

        n, d = X.shape
        g, m = cluster

        alpha_1, alpha_2 = alpha

        pis = self.initialization_theta(g, initialization_theta)
        rho = self.initialization_theta(m, initialization_theta)

        # initialisation
        Z_hat = self.initialization_cluster(n, g, initialization_cluster)
        W_hat = self.initialization_cluster(d, m, initialization_cluster)


        log_likelihoods = []

        for _ in range(max_iter):

            #E-step for rows
            S = np.zeros((n, g))
            for k in range(g):
                for i in range(n):
                    S[i,k] = pis[k]*np.prod([np.prod([(norm(mus[k,l], sigmas[k,l]).pdf(X[i,j])) if W_hat[j,l]==1 else 1 for j in range(d)]) for l in range(m)])
            S /= S.sum(axis=1)[:, np.newaxis]
            
            # T-step and C-step for rows
            Z_hat = np.zeros_like(S)
            ind = np.where(np.max(S, axis=1) > np.sort(np.max(S, axis=1))[int(np.ceil(n * alpha_1))])[0] # observations index not trimmed
            Z_hat[ind, np.argmax(S, axis=1)[ind]] = 1
            

            # E-step for columns
            T = np.zeros((d, m))
            for l in range(m):
                for j in range(d):
                    T[j,l] = rho[l]*np.prod([np.prod([(norm(mus[k,l], sigmas[k,l]).pdf(X[i,j])) if Z_hat[i,k]==1 else 1 for i in range(n)]) for k in range(g)])
            T /= T.sum(axis=1)[:, np.newaxis]

            # T-step and C-step for columns
            W_hat = np.zeros_like(T)
            ind = np.where(np.max(T, axis=1) > np.sort(np.max(T, axis=1))[int(np.ceil(d * alpha_2))])[0] # variables index not trimmed
            W_hat[ind, np.argmax(T, axis=1)[ind]] = 1

            # M-step
            mus = np.zeros((g, m))
            sigmas = np.zeros((g, m))
            for k in range(g):
                for l in range(m):
                    normalization = np.sum([S[i,k] for i in range(n)])*np.sum([T[j, l] for j in range(d)])
                    if normalization == 0:
                        raise ValueError('Initilization led to a cluster with no element')
                    mus[k,l] = np.sum([S[i,k]*T[j,l]*X[i,j] for i in range(n) for j in range(d)])
                    sigmas[k,l] = np.sum([S[i,k]*T[j,l]*(X[i,j]-mus[k,l])**2 for i in range(n) for j in range(d)])
                    mus[k,l] /= normalization
                    sigmas[k,l] /= normalization


            pis = Z_hat.sum(axis=0)/n
            rho = W_hat.sum(axis=0)/d

            # log likelihood
            log_likelihood = 0
            log_likelihood += np.sum([np.log(pis[k]) if Z_hat[i,k]==1 else 0 for i in range(n) for k in range(g)])
            log_likelihood += np.sum([np.log(rho[l]) if W_hat[j,l]==1 else 0 for j in range(d) for l in range(m)])
            for k in range(g):
                for l in range(m):
                    for i in range(n):
                        for j in range(d):
                            if Z_hat[i,k]==1 and W_hat[j,l]==1:
                                log_likelihood += np.log(norm(mus[k,l], sigmas[k,l]).pdf(X[i,j]))
            log_likelihoods.append(log_likelihood)

            if len(log_likelihoods)>2 and np.abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
                break    
        else:
            print('The algorithm did not converge')

        return {
            'log_likelihoods': log_likelihoods,
            'clusters_lignes': np.argmax(np.hstack((np.zeros((n, 1)), Z_hat)), axis=1), #add a column of zeros to put the outliers in the cluster zeros
            'clusters_colonnes': np.argmax(np.hstack((np.zeros((d, 1)), W_hat)), axis=1) #add a column of zeros to put the outliers in the cluster zeros
        }

    def initialization_theta(self, cluster, initialization):
        if initialization == 'random':
            theta = np.random.rand(cluster)
            theta /= theta.sum()
        elif initialization == 'uniform':
            theta = np.ones(cluster) / cluster
        else:
            raise ValueError('Initialization not recognized')
        return theta
    
    def initialization_cluster(self, nbr_data, cluster, initialization):
        if initialization == 'random':
            cluster_ = np.random.randint(0, cluster, nbr_data)
            cluster_ = np.eye(cluster)[cluster_]
            if np.any(cluster_.sum(axis=0)==0) or np.any(cluster_.sum(axis=1)==0): # check if there is an empty cluster
                return self.initialization_cluster(nbr_data, cluster, initialization)
        elif initialization == 'uniform':
            cluster_ = np.zeros((nbr_data, cluster))
            block = nbr_data//cluster
            for i in range(cluster):
                cluster_[i*block:(i+1)*block, i] = 1
            cluster_[nbr_data-block:,-1] = 1
        else:
            raise ValueError('Initialization not recognized')
        return cluster_

# Application

In [ ]:
TBCEM = TrimmedBlockCEM().auto_apply(df, (3, 2), (1/8, 0), epochs=1000)

In [ ]:
print(TBCEM)